In [2]:
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("data/covid/france/chiffres-cles.csv", sep=",")

In [9]:
df.columns

Index(['date', 'granularite', 'maille_code', 'maille_nom', 'cas_confirmes',
       'cas_ehpad', 'cas_confirmes_ehpad', 'cas_possibles_ehpad', 'deces',
       'deces_ehpad', 'reanimation', 'hospitalises', 'gueris', 'depistes',
       'source_nom', 'source_url', 'source_archive', 'source_type'],
      dtype='object')

In [10]:
df.dtypes

date                    object
granularite             object
maille_code             object
maille_nom              object
cas_confirmes          float64
cas_ehpad              float64
cas_confirmes_ehpad    float64
cas_possibles_ehpad    float64
deces                  float64
deces_ehpad            float64
reanimation            float64
hospitalises           float64
gueris                 float64
depistes               float64
source_nom              object
source_url              object
source_archive          object
source_type             object
dtype: object

In [19]:
df[df["maille_code"] == "FRA"][["date", "cas_confirmes", "source_nom"]]

,date,cas_confirmes,source_nom
13,2020-01-24,3.0,Santé publique France
14,2020-01-24,3.0,Ministère des Solidarités et de la Santé
22,2020-01-25,3.0,Santé publique France
27,2020-01-26,3.0,Santé publique France
33,2020-01-27,3.0,Santé publique France
...,...,...,...
8938,2020-05-04,NaN,OpenCOVID19-fr
9058,2020-05-05,132967.0,Ministère des Solidarités et de la Santé
9059,2020-05-05,NaN,OpenCOVID19-fr
9179,2020-05-06,137150.0,Ministère des Solidarités et de la Santé


Il y a des valeurs NAN selon les sources, on souhaite savoir s'il y a des données conflictuelles

In [51]:
for d in df["date"].unique():
    s = df[np.logical_and(df["maille_code"] == "FRA", df["date"] == d)]["cas_confirmes"].dropna()
    if s.size > 1:
        if s.drop_duplicates().size > 1:
            print(d)
            print(s)

2020-02-07
91    11.0
92     6.0
Name: cas_confirmes, dtype: float64
2020-02-25
157    12.0
158    14.0
Name: cas_confirmes, dtype: float64
2020-02-26
170    17.0
171    18.0
Name: cas_confirmes, dtype: float64
2020-02-28
194    38.0
195    57.0
Name: cas_confirmes, dtype: float64
2020-02-29
231     73.0
232    100.0
Name: cas_confirmes, dtype: float64
2020-03-02
270    178.0
271    191.0
Name: cas_confirmes, dtype: float64
2020-03-14
1603    4499.0
1604    4500.0
Name: cas_confirmes, dtype: float64


On trouve quelques cas conflictuels. Pour résoudre le souci on prendra la valeur maximum des deux à chaque fois. On supprime donc toutes les lignes à cas confirmes NAN, et pour les cas à plusieurs sources on prendra le max.

In [60]:
# on ne s'intéresse qu'à la france entière pour l'instant
df_copy = df.copy()
df_copy = df_copy[df_copy["maille_code"] == "FRA"]

In [90]:
df_copy.drop(df_copy[df_copy["cas_confirmes"].isna()].index)

,date,granularite,maille_code,maille_nom,cas_confirmes,cas_ehpad,cas_confirmes_ehpad,cas_possibles_ehpad,deces,deces_ehpad,reanimation,hospitalises,gueris,depistes,source_nom,source_url,source_archive,source_type
13,2020-01-24,pays,FRA,France,3.0,NaN,NaN,NaN,0.0,NaN,NaN,3.0,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200124202351/htt...,sante-publique-france
14,2020-01-24,pays,FRA,France,3.0,NaN,NaN,NaN,0.0,NaN,NaN,3.0,NaN,NaN,Ministère des Solidarités et de la Santé,https://solidarites-sante.gouv.fr/IMG/pdf/2001...,https://web.archive.org/web/20200322183046/htt...,ministere-sante
22,2020-01-25,pays,FRA,France,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200126024644/htt...,sante-publique-france
27,2020-01-26,pays,FRA,France,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200127144431/htt...,sante-publique-france
33,2020-01-27,pays,FRA,France,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200128153608/htt...,sante-publique-france
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8695,2020-05-02,pays,FRA,France,130979.0,70688.0,33271.0,NaN,15487.0,9273.0,3827.0,25827.0,50562.0,NaN,Ministère des Solidarités et de la Santé,NaN,NaN,ministere-sante
8816,2020-05-03,pays,FRA,France,131287.0,70767.0,33361.0,NaN,15583.0,9312.0,3819.0,25815.0,50784.0,NaN,Ministère des Solidarités et de la Santé,NaN,NaN,ministere-sante
8937,2020-05-04,pays,FRA,France,131863.0,71390.0,33791.0,NaN,15826.0,9375.0,3696.0,25548.0,51371.0,NaN,Ministère des Solidarités et de la Santé,NaN,NaN,ministere-sante
9058,2020-05-05,pays,FRA,France,132967.0,71692.0,34108.0,NaN,16060.0,9471.0,3430.0,24775.0,52736.0,NaN,Ministère des Solidarités et de la Santé,NaN,NaN,ministere-sante


In [40]:
s - set([float('nan')])

{nan, 132967.0}